<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/CodeBERT_1_N_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/Last.csv')

In [6]:
df

,CWE ID,code,vul,lang,labels,Unnamed: 0
0,CWE-20,unsigned long convert_rip_to_linear(struct tas...,1,c,2,NaN
1,CWE-20,static unsigned long convert_eip_to_linear(str...,1,c,2,NaN
2,CWE-399,static int pwc_video_close(struct inode *inode...,1,cpp,9,NaN
3,CWE-399,static void usb_pwc_disconnect(struct usb_inte...,1,c,9,NaN
4,CWE-20,asmlinkage long sys_setrlimit(unsigned int res...,1,cpp,2,NaN
...,...,...,...,...,...,...
18611,CWE-416,void func() {\n\t wchar_t * data; /* Ini...,1,cpp,10,NaN
18612,CWE-416,void func() {\n\t wchar_t * data; /* Ini...,1,cpp,10,NaN
18613,CWE-121,void func() {\n\t wchar_t * data; other_...,1,c,5,NaN
18614,CWE-121,static void func(wchar_t * data) {\n\t {\n...,1,c,5,NaN


In [7]:
df.columns

Index(['CWE ID', 'code', 'vul', 'lang', 'labels', 'Unnamed: 0'], dtype='object')

In [8]:
df = df[['CWE ID','code', 'labels']]

In [9]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
batch_size = 16
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    preprocess_df = df.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df = preprocess_df.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df = preprocess_df.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df = preprocess_df.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df = df.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [29]:
preprocess(df,"preprocess.csv")
# 'code' 컬럼을 사용하여 데이터셋 생성
dataset = load_dataset("csv", data_files="preprocess.csv")['train']

# 데이터셋에서 'code' 컬럼에 접근하여 tokenizer를 적용
encoded_dataset = dataset.map(lambda example: tokenizer(example['code'], padding=True, truncation=True, return_tensors='pt'), batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/18616 [00:00<?, ? examples/s]

In [13]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [14]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['CWE ID', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13031
    })
    test: Dataset({
        features: ['CWE ID', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5585
    })
})

In [15]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [18]:
from transformers import TrainingArguments, Trainer
import pandas as pd
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import TrainerCallback, TrainerControl, TrainerState

class EarlyStoppingCallback(TrainerCallback):
    """ 조기 종료 콜백 구현 """
    def __init__(self, early_stopping_patience=5):
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = None
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 평가 단계에서의 손실 가져오기
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')
        if eval_loss is not None:
            if self.best_loss is None or eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            # 조기 종료 조건 체크
            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True

from transformers import Trainer, TrainingArguments

# 데이터셋과 데이터로더 분할
train_dataset, test_dataset = encoded_dataset['train'], encoded_dataset['test']

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)

# Trainer 객체 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # 수정된 부분: train_dataset을 사용
    eval_dataset=test_dataset,    # 수정된 부분: test_dataset을 사용
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(5)]
)



In [19]:
gc.collect()
torch.cuda.empty_cache()

In [20]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9,F1 Class 10
500,1.418900,0.796148,0.691137,0.691137,0.700289,0.685675,0.691137,0.888116,0.419375,0.410377,0.944911,0.897016,0.878906,0.274882,0.937093,0.851632,0.290116,0.750000
1000,0.684100,0.601078,0.783169,0.783169,0.782255,0.768760,0.783169,0.920082,0.625887,0.461682,0.947858,0.897485,0.876209,0.694981,0.940265,0.863874,0.478036,0.750000
1500,0.550700,0.506865,0.811996,0.811996,0.814818,0.799535,0.811996,0.960742,0.678600,0.620237,0.944898,0.860524,0.882479,0.708434,0.926829,0.885073,0.567515,0.759551
2000,0.417200,0.504826,0.846374,0.846374,0.846939,0.834539,0.846374,0.962446,0.717270,0.712906,0.947566,0.884340,0.888273,0.762800,0.953744,0.916898,0.651079,0.782609
2500,0.313300,0.450643,0.854252,0.854252,0.855222,0.845879,0.854252,0.966702,0.740289,0.720000,0.948118,0.891150,0.854949,0.770398,0.951965,0.931694,0.712903,0.816495
3000,0.266000,0.545002,0.859266,0.859266,0.858443,0.845874,0.859266,0.966074,0.745921,0.765694,0.950514,0.885772,0.886216,0.783582,0.944934,0.900524,0.672727,0.802661
3500,0.200900,0.634626,0.856043,0.856043,0.857665,0.848654,0.856043,0.960265,0.757370,0.756911,0.948118,0.856401,0.873807,0.769084,0.931677,0.934426,0.741405,0.805726
4000,0.220700,0.576571,0.862489,0.862489,0.862572,0.853508,0.862489,0.967811,0.756400,0.756074,0.948118,0.847943,0.880698,0.795899,0.956522,0.912467,0.759644,0.807018
4500,0.172400,0.587616,0.868756,0.868756,0.868841,0.860478,0.868756,0.961824,0.759259,0.773471,0.931382,0.887795,0.886940,0.807302,0.956427,0.937838,0.739496,0.823529
5000,0.151900,0.532205,0.876634,0.876634,0.876939,0.869296,0.876634,0.964693,0.787557,0.805755,0.934597,0.885246,0.886756,0.807339,0.929520,0.939597,0.793750,0.827443


No improvement in 5 evaluation steps. Stopping training early.


TrainOutput(global_step=5000, training_loss=0.439614176940918, metrics={'train_runtime': 6212.7136, 'train_samples_per_second': 2097.473, 'train_steps_per_second': 104.946, 'total_flos': 2.6296654331249664e+16, 'train_loss': 0.439614176940918, 'epoch': 7.668711656441718})

In [28]:
print(df.columns)


Index(['CWE ID', 'code', 'labels'], dtype='object')


In [57]:
# 평가 데이터셋에 대한 예측 수행
predictions = trainer.predict(test_dataset)

In [59]:
# 예측된 레이블과 실제 레이블 추출
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# 각 클래스별 F1 스코어 계산
class_f1_scores = f1_score(true_labels, predicted_labels, average=None)

# 클래스별 F1 스코어 출력
for class_id, f1_score_value in enumerate(class_f1_scores):
    print(f"Class {class_id} F1 Score: {f1_score_value}")

Class 0 F1 Score: 0.966702470461869
Class 1 F1 Score: 0.7402894135567403
Class 2 F1 Score: 0.72
Class 3 F1 Score: 0.948118006103764
Class 4 F1 Score: 0.8911495422177008
Class 5 F1 Score: 0.8549488054607509
Class 6 F1 Score: 0.7703984819734344
Class 7 F1 Score: 0.9519650655021833
Class 8 F1 Score: 0.9316939890710383
Class 9 F1 Score: 0.7129032258064516
Class 10 F1 Score: 0.8164948453608248


In [61]:
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score, classification_report
import pickle  # 추가: 모델 저장을 위한 pickle 라이브러리
import numpy as np

# 모델의 로짓과 레이블 추출
logits = predictions.predictions
labels = predictions.label_ids

# XGBoost 입력 데이터 준비
xgboost_input = logits  # 이미 numpy.ndarray 형식이므로 그대로 사용

# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=11)

# XGBoost 모델 학습
xgb_model.fit(xgboost_input, labels)

# 예측 결과 확인
y_pred = xgb_model.predict(xgboost_input)

# 정확도 계산
accuracy = accuracy_score(labels, y_pred)
print(f"XGBoost 모델 정확도: {accuracy}")

# F1 스코어 계산
f1_macro = f1_score(labels, y_pred, average='macro')
f1_micro = f1_score(labels, y_pred, average='micro')
f1_weighted = f1_score(labels, y_pred, average='weighted')

# 각 클래스별 F1 스코어 계산
class_f1_scores = f1_score(labels, y_pred, average=None, labels=np.unique(labels))

print(f"XGBoost 모델 Macro F1 Score: {f1_macro}")
print(f"XGBoost 모델 Micro F1 Score: {f1_micro}")
print(f"XGBoost 모델 Weighted F1 Score: {f1_weighted}")

# 각 클래스별 F1 스코어 출력
for class_id, f1_score_value in enumerate(class_f1_scores):
    print(f"Class {class_id} F1 Score: {f1_score_value}")

# 각 클래스별 Precision, Recall, F1 Score, Support 출력
class_report = classification_report(labels, y_pred)
print("XGBoost 모델 클래스별 성능 보고서:")
print(class_report)

# 모델 저장
model_path = 'xgboost_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(xgb_model, f)

print(f"XGBoost 모델이 {model_path}에 저장되었습니다.")


XGBoost 모델 정확도: 0.9790510295434198
XGBoost 모델 Macro F1 Score: 0.9793636689834607
XGBoost 모델 Micro F1 Score: 0.9790510295434198
XGBoost 모델 Weighted F1 Score: 0.9789645036957934
Class 0 F1 Score: 0.9972958355868036
Class 1 F1 Score: 0.9959644874899112
Class 2 F1 Score: 0.9982486865148862
Class 3 F1 Score: 0.9563602599814298
Class 4 F1 Score: 0.9362880886426592
Class 5 F1 Score: 0.9216494845360825
Class 6 F1 Score: 0.9958932238193018
Class 7 F1 Score: 0.9755058572949946
Class 8 F1 Score: 0.9973544973544973
Class 9 F1 Score: 0.9984399375975038
Class 10 F1 Score: 1.0
XGBoost 모델 클래스별 성능 보고서:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       927
           1       1.00      0.99      1.00       621
           2       1.00      1.00      1.00       571
           3       0.92      1.00      0.96       515
           4       0.94      0.94      0.94       542
           5       0.96      0.89      0.92       503
           6       0.99     

In [56]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Trainer를 통해 훈련된 모델을 사용하여 예측 생성
def generate_predictions(trainer, dataset):
    predictions = trainer.predict(dataset)
    logits = predictions.predictions
    predicted_labels = np.argmax(logits, axis=-1)
    return predicted_labels

# 훈련된 모델로 테스트 데이터에 대한 예측 생성
test_predictions = generate_predictions(trainer, test_dataset)

# 테스트 데이터의 실제 레이블 가져오기
test_labels = test_dataset['labels']

# XGBoost를 위한 데이터셋 준비 (훈련 데이터와 검증 데이터로 분할)
train_data, eval_data, train_labels, eval_labels = train_test_split(test_predictions.reshape(-1, 1), test_labels, test_size=0.2, random_state=42)

# XGBoost 모델 초기화 및 학습
boosting_model = xgb.XGBClassifier(objective='multi:softmax', num_class=11, eval_metric='mlogloss')
boosting_model.fit(train_data, train_labels)

# 모델 성능 평가를 위해 검증 데이터에 대한 예측 생성
eval_predictions = boosting_model.predict(eval_data.reshape(-1, 1))

# 정확도 계산
accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"XGBoost 모델 정확도: {accuracy}")

# Weighted F1 스코어 계산
weighted_f1 = f1_score(eval_labels, eval_predictions, average='weighted')
print(f"XGBoost 모델 Weighted F1 Score: {weighted_f1}")

# 각 클래스에 대한 세부적인 F1 스코어 출력
class_f1_scores = f1_score(eval_labels, eval_predictions, average=None)
print("XGBoost 모델 각 클래스별 F1 Score:")
for i, score in enumerate(class_f1_scores):
    print(f"Class {i}: {score}")

# 분류 리포트 출력
print("XGBoost 모델 분류 리포트:")
print(classification_report(eval_labels, eval_predictions))


XGBoost 모델 정확도: 0.8343777976723367
XGBoost 모델 Weighted F1 Score: 0.8366570269145183
XGBoost 모델 각 클래스별 F1 Score:
Class 0: 0.967741935483871
Class 1: 0.6978417266187051
Class 2: 0.7136150234741784
Class 3: 0.9508196721311475
Class 4: 0.8488372093023255
Class 5: 0.8305084745762712
Class 6: 0.7155963302752292
Class 7: 0.9339622641509434
Class 8: 0.927536231884058
Class 9: 0.7547169811320755
Class 10: 0.8333333333333334
XGBoost 모델 분류 리포트:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       170
           1       0.66      0.75      0.70       130
           2       0.78      0.66      0.71       115
           3       1.00      0.91      0.95        96
           4       0.97      0.75      0.85        97
           5       0.72      0.99      0.83        99
           6       0.67      0.76      0.72       102
           7       0.97      0.90      0.93       110
           8       1.00      0.86      0.93        74
           9       0.

In [ ]:
trainer.save_model('model_codebert(LEE)')

NameError: name 'trainer' is not defined